<a href="https://colab.research.google.com/github/IlyaBurlak/DA-and-IR/blob/main/LAB_1V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установка необходимых библиотек

In [ ]:
%%capture
!pip install feedparser yake networkx python-louvain matplotlib plotly
!pip install transformers tqdm

# Импорт библиотек

In [ ]:
import feedparser
import urllib.request as libreq
import pandas as pd
from transformers import pipeline
import networkx as nx
from itertools import combinations, chain
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from operator import itemgetter
import plotly.graph_objects as go
from community import community_louvain
from tqdm import tqdm
import yake

# Сбор данных с arXiv

In [ ]:
def fetch_articles(query, max_results=300):
    url = f'http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results={max_results}'
    with libreq.urlopen(url) as url:
        data = url.read()
    return data

data = fetch_articles('python', 300)
docs = feedparser.parse(data)

pubs = []
for entry in docs["entries"]:
    data = {
        "title": entry['title'],
        "abstract": entry['summary'],
        "authors": [author['name'] for author in entry['authors']],
        "tags": [tag['term'] for tag in entry.get('tags', [])]
    }
    pubs.append(data)

df = pd.DataFrame(pubs)
print(f"Загружено статей: {len(df)}")

# Извлечение ключевых слов с помощью YAKE

In [ ]:
tqdm.pandas()

# Функция для извлечения ключевых слов
def extract_keywords(text):
    keyword_extractor = yake.KeywordExtractor(lan="en", n=2, dedupLim=0.5, top=5)
    keywords = keyword_extractor.extract_keywords(text)
    return [kw[0] for kw in keywords if 'python' not in kw[0].lower()]

# Применение функции к абстрактам
df['keywords'] = df['abstract'].progress_apply(extract_keywords)

# Просмотр результатов
print("Пример извлеченных ключевых слов:")
print(df[['title', 'keywords']].head(3))

# Построение графа ключевых слов


In [ ]:
# Создание списка ключевых слов
klist = df["keywords"].tolist()

# Создание ребер (связей между ключевыми словами)
edges = list(chain(*[list(combinations(words, 2)) for words in klist]))
edges = [tuple(sorted(edge)) for edge in edges]
weighted_edges = [(edge[0], edge[1], {"weight": edges.count(edge)}) for edge in set(edges)]

print(f"Всего уникальных ребер: {len(weighted_edges)}")

# Создание графа
G = nx.Graph()
G.add_edges_from(weighted_edges)

# Базовая визуализация графа
plt.figure(figsize=(12, 8))
nx.draw(G, with_labels=False, node_size=30, alpha=0.6)
plt.title("Граф ключевых слов")
plt.show()

# Кластеризация и анализ сообществ


In [ ]:
# Выявление сообществ с помощью алгоритма Лувена
communities = community_louvain.best_partition(G)
modularity = community_louvain.modularity(communities, G)
print(f"Модулярность кластеризации: {modularity:.3f}")

# Визуализация топ-3 сообществ
top_comms = pd.Series(communities).value_counts().head(3).index
nodes = [node for node, comm in communities.items() if comm in top_comms]
S = G.subgraph(nodes)

node_colors = [communities[node] for node in S.nodes()]
plt.figure(figsize=(12, 8))
nx.draw(S, node_color=node_colors, cmap=plt.cm.Set3, with_labels=True, font_size=8)
plt.title("Топ-3 сообщества ключевых слов")
plt.show()

# Интерактивная визуализация графа


In [ ]:
# Подготовка данных для интерактивной визуализации
pos = nx.spring_layout(S, iterations=50, seed=42)
edge_x, edge_y = [], []
for edge in S.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

node_x = [pos[node][0] for node in S.nodes()]
node_y = [pos[node][1] for node in S.nodes()]

# Создание интерактивного графа
fig = go.Figure()

# Добавление ребер
fig.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
))

# Создание правильной цветовой шкалы
colorscale = [
    [0, 'rgb(141,211,199)'],
    [0.5, 'rgb(255,255,179)'],
    [1, 'rgb(190,186,218)']
]

# Добавление узлов
fig.add_trace(go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=list(S.nodes()),
    textposition="bottom center",
    marker=dict(
        size=10,
        color=node_colors,
        colorscale=colorscale,
        showscale=False
    )
))

fig.update_layout(
    title="Интерактивная визуализация сообществ",
    showlegend=False,
    hovermode='closest'
)
fig.show()

# Анализ центральности ключевых слов


In [ ]:
# Расчет и отображение центральных ключевых слов
centrality = nx.degree_centrality(S)
top_nodes = sorted(centrality.items(), key=itemgetter(1), reverse=True)[:10]

print("Наиболее центральные ключевые слова:")
for node, centr in top_nodes:
    print(f"{node}: {centr:.3f}")

# Поиск похожих статей


In [ ]:
# Функция для поиска похожих статей по ключевым словам
def find_similar_articles(article_id, df, top_n=3):
    target_keywords = set(df.iloc[article_id]['keywords'])
    similarities = []

    for idx, row in df.iterrows():
        if idx == article_id:
            continue
        other_keywords = set(row['keywords'])
        if len(target_keywords | other_keywords) == 0:
            continue
        similarity = len(target_keywords & other_keywords) / len(target_keywords | other_keywords)
        similarities.append((idx, similarity))

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

# Пример использования
article_id = 0
similar = find_similar_articles(article_id, df)

print(f"Статья для сравнения: {df.iloc[article_id]['title'][:50]}...")
print("\nБлижайшие статьи:")
for art_id, sim in similar:
    print(f"Статья {art_id}: {df.iloc[art_id]['title'][:50]}... ")

In [ ]:
# Сохранение DataFrame с ключевыми словами
df.to_csv("arxiv_python_articles.csv", index=False)
print("Результаты сохранены в файл arxiv_python_articles.csv")